PKL reader

In [ ]:
from Code.Utils.util_methods import NNUtils
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from tqdm import tqdm


cwd = os.getcwd()
base = f'{cwd}/../..'

## Give the path to the pkl file

In [ ]:
FILEPATH = 'YOUR PATH'
FILTER_COLUMNS = ['Name', 'Form', 'Mw', 'SMILES'] # example
#FILTER_COLUMNS = None

## Extract the pkl file into a df

In [ ]:
df = NNUtils.read_big_csv(FILEPATH)

In [ ]:
if FILTER_COLUMNS:
    df = df[FILTER_COLUMNS]
print(df.shape)
df

In [ ]:
df['smile_form_match'] = 99 # -2 means contains . and no match -1 means incorrect form 0 means no match 1 means match
df['new_form'] = df['Form']

c=0

for index in tqdm(range(df.shape[0])):

    # convert smiles to a molecule object
    if df.loc[index, 'SMILES'] is None:
        continue
    
    mol = Chem.MolFromSmiles(df.loc[index, 'SMILES'])
    if "." in df.loc[index, 'SMILES']:
        c+=1
    
    # check if the molecule is valid
    if mol:
        # calculate the molecular formula
        molecular_formula = rdMolDescriptors.CalcMolFormula(mol)
        if molecular_formula == df.loc[index, 'Form']:
            df.loc[index, 'smile_form_match'] = 1
        elif "." in df.loc[index, 'SMILES']:
            df.loc[index, 'smile_form_match'] = -2
        else:
            df.loc[index, 'smile_form_match'] = 0
            df.loc[index, 'new_form'] = molecular_formula
    else:
        df.loc[index, 'smile_form_match'] = -1
        
print(f'{c} molecules contain .')

In [ ]:
df[df['smile_form_match'] == -2]

In [ ]:
df[df['smile_form_match'] == 1]